In [40]:
import cv2
import numpy as np
import math

In [9]:
cam_id = 0
api = cv2.CAP_DSHOW
res = (1280, 720)
capture = cv2.VideoCapture(cam_id, api)
focus = 0
capture.set(28, focus)
capture.set(3, 1280)
capture.set(4, 720)

True

In [10]:
def calibrate(cam_img):
    cv2.circle(cam_img,(1280//2, 360), 5, (255, 255, 0), 2)
    cv2.line(cam_img, (1280//2, 0), (1280//2, 720), (0, 255, 0), 1)
    cv2.line(cam_img, (0, 360-37//2), (1280, 360-37//2), (255, 0, 0), 1)
    cv2.line(cam_img, (0, 360), (1280, 360), (255, 0, 0), 1)

def calc_circles(image):
    minr = 8
    maxr = 20
    edges = cv2.Canny(image, 100, 150)
    param1 = 200
    param2 = 30
    mindist = 10
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, mindist, param1=param1, param2=param2, maxRadius=maxr, minRadius=minr)
    edges2 = edges.copy()
    if circles is not None:
        for ind, i in enumerate(circles[0,:].astype(int)):
            cv2.circle(image,(i[0], i[1]), i[2], (0, 255, 0), 2)
            edges1 = edges.copy()
            #cv2.circle(edges1,(i[0], i[1]), i[2], (255, 255, 0), 2)
            #cv2.circle(edges2,(i[0], i[1]), i[2], (255, 255, 0), 1)
            #print(edges.shape)
            #print(f'{ind} - {abs(edges1-edges).sum()//255}, r - {i[2]}, xy - {i[:2]}')
        #return edges2
        return i[:2]
    

In [45]:
num_of_samples = 10
ind = 0
avg = np.array([0, 0])
show_lines = False
while True:
    _, cam_img = capture.read()
    center = calc_circles(cam_img)
    if center is not None:
        ind += 1
        avg += center
        if ind == num_of_samples:
            avg //= 10
            print(avg)
            
            avg = avg.astype(float)/10
            avg = np.array([0, 0])
            ind = 0
    
    if show_lines:
        calibrate(cam_img)
    
    #cv2.imshow('w', cam_img)
    
    imgray = cv2.cvtColor(cam_img, cv2.COLOR_BGR2GRAY)
    ret, edges = cv2.threshold(imgray, 150, 255, 0)
    #edges = cv2.Canny(cam_img, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = np.vectorize(cv2.contourArea)(contours)
    contours = np.array(contours)
    contours = contours[(areas > 500) & (areas < 2000)]
    
    rects = [cv2.minAreaRect(cnt) for cnt in contours]
    boxes = [np.int0(cv2.boxPoints(rect)) for rect in rects]
    
    good_boxes = []
    for box in boxes:
        M = cv2.moments(box)
        r = 360
        fig_center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        center = (640, 360)
        if math.dist(center, fig_center) < 380:
            good_boxes.append(box)
            cv2.circle(cam_img, fig_center, 3, (255, 255, 0), -1)
    cv2.drawContours(cam_img,good_boxes,-1,(0,0,255),2)
    cv2.imshow('w', cam_img)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('c'):
        show_lines = not show_lines
cv2.destroyAllWindows()

C:\Users\bkmz1\AppData\Local\Temp/ipykernel_1292/2253864280.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  contours = np.array(contours)


In [16]:
help(np.int0)

Help on class int64 in module numpy:

class int64(signedinteger)
 |  Signed integer type, compatible with C ``long long``.
 |  
 |  :Character code: ``'q'``
 |  :Canonical name: `numpy.longlong`
 |  :Alias on this platform (Windows AMD64): `numpy.int64`: 64-bit signed integer (``-9_223_372_036_854_775_808`` to ``9_223_372_036_854_775_807``).
 |  :Alias on this platform (Windows AMD64): `numpy.intp`: Signed integer large enough to fit pointer, compatible with C ``intptr_t``.
 |  
 |  Method resolution order:
 |      int64
 |      signedinteger
 |      integer
 |      number
 |      generic
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 | 

In [5]:
#cv2.imwrite(r'C:\Users\bkmz1\_notebooks\chess\images\image_001.jpg', cam_img)

True

In [6]:
#capture.release()